In [ ]:
# !pip install prince

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# 차원축소
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
import prince


In [ ]:
df = pd.read_excel('../data/preprocess_data.xlsx')

In [ ]:
categorical_variable = ['5일장', '복합장', '상설장','공중화장실보유여부', '주차장보유여부','농산물이 주요품목']
df_number = df.drop(categorical_variable, axis = 1)
scaled_data = StandardScaler().fit_transform(np.log1p(df_number))
data_scale_log=pd.DataFrame(data=scaled_data, columns=df_number.columns)

## 차원축소

- 시장특성, 주변시설특성, 소비자특성으로 나뉘어 차원축소를 진행한다.
- 연속형 변수는 PCA, 명목형 변수는 MCA기법으로 차원축소를 진행한다.
- 이때 설명력은 90%로 한다.
- 유동인구 데이터의 경우, "통계청"을 방문하여 "공개불가 데이터"를 활용하여 진행하여 PPT와 다소 차이가 있을 수 있다.

In [ ]:
categorical = ['5일장', '복합장', '상설장','공중화장실보유여부', '주차장보유여부','농산물이 주요품목']
continuous = ['점포수','운영기간', '대형마트', '주차장', '대중교통', '학교', '편의점', '주유소,충전소', 
              '문화시설', '관광명소', '음식점', '카페', '공시지가', '행정동 인구', '행정시구 인구', 
              '시도별 소득월액', '시구 미성년자', '시구 젊은청년', '시구 소득인구', '시구 노년인구',
              '동별 미성년자', '동별 젊은청년', '동별 소득인구','동별 노년인구']

df_categorical = df[categorical] 
df_continuous = df[continuous] 

### 시장특성

다음 분석으로부터 다음의 결과를 도출 할 수 있다.  
시장특성의 연속형 변수의 경우, 통계청 데이터인 유동인구 데이터가 본 과정에서는 누락되어 있어 다소 차이가 있다.

연속형 변수 
- 1번 주성분
    - 점포수가 많고 공시지가가 높으며 운영기간이 짧음
    - 규모가 있는 신설시장
- 2번 주성분
    - 점포수가 적고, 운영기간도 짧음
    - 규모가 작은 신설시장
- 3번 주성분: 
    - 점포수가 많고, 공시지가가 낮으며, 운영기간이 짧음 
    - 도시 외각 지역의 규모가 큰 신설시장

명목형 변수
- 1번 성분
    - 5일장과 복합장이고 주차장을 보유한 시장
    - 주차장을 보유한 5일장, 복합장
- 2번 성분
    - 복합장 성분이 큼
    - 복합장 시장
- 3번 성분
    - 농산물이 주요 품목이고 공중화장실과 주차장이 없음
    - 편의시설이 없는 농산물 시장
- 4번 성분
    - 주차장은 없으나 공중화장실을 보유함.
    - 주차장은 없지만 화장실을 보유한 시장    


#### 시장특성 - 연속형 변수



In [ ]:
market_feature = ['점포수', '운영기간', '공시지가']

pca = PCA(n_components=0.9)
market_scale_data = data_scale_log[market_feature]
pca.fit(market_scale_data)
pca_market = pd.DataFrame((pca.transform(market_scale_data)))

num_of_principal = pca_market.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 3


In [ ]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=market_scale_data.columns)

주성분의 설명력
제 1주성분 : 0.47233351493546794
제 2주성분 : 0.29324421240316306
제 3주성분 : 0.23442227266136903


,점포수,운영기간,공시지가
0,0.571409,-0.509901,0.643034
1,-0.600783,-0.793687,-0.095500
2,0.559064,-0.331754,-0.759860


#### 시장특성 - 명목형 변수

In [ ]:
mca = prince.MCA(n_components=4)
mca = mca.fit(df_categorical)

mca_eigen = mca.eigenvalues_
mca_score = np.array(mca_eigen)/sum(mca_eigen)
print('mca의 설명력', round(mca_score[0], 3), round(mca_score[1], 3), round(mca_score[2], 3), round(mca_score[3], 3))

mca.column_coordinates(df_categorical)

mca의 설명력 0.385 0.346 0.191 0.078


,0,1,2,3
5일장,1.683236,-1.439126,-0.089836,0.335414
복합장,1.298242,2.355810,0.009018,0.391115
상설장,-0.731257,-0.075435,-0.051919,0.182308
공중화장실보유여부,0.012938,0.000428,-0.205284,0.148783
주차장보유여부,0.140530,0.054964,-0.311220,-0.419484
농산물이 주요품목,0.060906,-0.029310,0.903976,-0.122447


In [ ]:
df_mca = mca.transform(df_categorical)

### 주변시설 특성

다음의 분석으로 부터, 다음의 결과를 이끌어 낼 수 있었다.

- 1번 주성분
    - 전체적으로 주성분 값이 낮음
    - 주변시설이 낙후된 시장
- 2번 주성분
    - 관광명소와 문화시설이 주위에 많고 대형마트가 없다.
    - 대형마트가 없는 관광지구시장
- 3번 주성분
    - 주변에 주유소,충전소가 많고 관광명소와 대형마트가 없다.
    - 주유소,충전소가 많고 대형마트가 없는 비관광지구시장
- 4번 주성분
    - 주변에 대중교통 시설이 많고 주유소, 충전소가 없다.
    - 대중교통 이용이 편리한 시장
- 5번 주성분
    - 주변에 학교가 많고, 대중교통 시설, 관광명소, 주유소,충전소가 많다.
    - 학교 주변 시장
- 6번 주성분
    - 문화시설이 많고 학교와 관광명소가 없다.
    - 학교, 관광명소와 떨어진 문화시설 주변시장



In [ ]:
infra_feature = ['대형마트', '주차장', '대중교통', '학교', '편의점', '주유소,충전소', '문화시설', '관광명소', '음식점', '카페']

pca = PCA(n_components=0.9)
infra_scale_data = data_scale_log[infra_feature]
pca.fit(infra_scale_data)
pca_infra = pd.DataFrame((pca.transform(infra_scale_data)))

num_of_principal = pca_infra.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 6


In [ ]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=infra_scale_data.columns)

주성분의 설명력
제 1주성분 : 0.5473909408982282
제 2주성분 : 0.12678620063118606
제 3주성분 : 0.08752356527033571
제 4주성분 : 0.07193393151090556
제 5주성분 : 0.061816178917672895
제 6주성분 : 0.05044514288822882


,대형마트,주차장,대중교통,학교,편의점,"주유소,충전소",문화시설,관광명소,음식점,카페
0,-0.135772,-0.391140,-0.270441,-0.311242,-0.405498,-0.219924,-0.316904,-0.078581,-0.412513,-0.411096
1,-0.446316,0.085210,-0.103628,-0.147402,-0.069199,-0.259577,0.393021,0.727432,-0.045401,0.056763
2,-0.799174,0.063263,0.085194,-0.106591,0.025141,0.477572,-0.004546,-0.331379,0.012303,0.002621
3,-0.260760,-0.055035,0.563255,0.091492,0.066676,-0.714085,-0.032848,-0.288121,0.034147,0.061052
4,-0.173906,0.183115,-0.740874,0.353674,0.091295,-0.341189,0.106724,-0.331635,0.105784,0.070277
5,0.180533,0.159544,-0.088408,-0.789286,0.045872,-0.113828,0.401126,-0.334990,0.108016,0.106381


### 소비자 특성

다음의 분석으로 부터, 다음의 결과를 이끌어 낼 수 있었다.

- 1번 주성분
    - 전반적으로 인구 수치가 음의값을 가진다.
    - 외각 지역의 시장
- 2번 주성분
    - 행정 시구인구와 행정동 인구가 반비례관계를 띈다.
    - 주거지에서 벗어난 시장

In [ ]:
consumer_feature = ['행정동 인구', '행정시구 인구', '시도별 소득월액', '시구 미성년자',
       '시구 젊은청년', '시구 소득인구', '시구 노년인구', '동별 미성년자', '동별 젊은청년', '동별 소득인구', '동별 노년인구']

pca = PCA(n_components=0.9)
consumer_scale_data = data_scale_log[consumer_feature]
pca.fit(consumer_scale_data)
pca_consumer = pd.DataFrame((pca.transform(consumer_scale_data)))

num_of_principal = pca_consumer.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 2


In [ ]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=consumer_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.6506768510549017
제 2주성분 : 0.2567898487125134


,0,1
행정동 인구,-0.306672,-0.338148
행정시구 인구,-0.323070,0.294338
시도별 소득월액,-0.254740,0.093304
시구 미성년자,-0.314333,0.298060
시구 젊은청년,-0.325548,0.282307
시구 소득인구,-0.323949,0.290569
시구 노년인구,-0.296912,0.296372
동별 미성년자,-0.277539,-0.367000
동별 젊은청년,-0.320794,-0.275098
동별 소득인구,-0.309084,-0.329438


## 차원축소 결과

In [ ]:
pca_mca=pd.concat([pca_market,df_mca,pca_infra,pca_consumer],axis=1)


In [ ]:
pca_mca.columns=['규모가 있는 신설시장',
'규모가 작은 신설시장',
'도시 외각 지역의 규모가 큰 신설시장',
 
'주차장을 보유한 5일장, 복합장',
'복합장 시장',                 
'편의시설이 없는 농산물 시장',
'화장실만 보유한 시장',                 
                 
'주변 시설이 낙후된 시장',
'대형마트가 없는 관광지구 시장',
'주유소,충전소가 많은 대형마트가 없는 비관광지구 시장',
'대중교통 이용이 편리한 시장',
'학교 주변 시장',
'학교, 관광명소와 떨어진 문화시설 주변 시장',

'외각지역의 시장',
'주거지에서 벗어난 시장']

pca_mca.head()

,규모가 있는 신설시장,규모가 작은 신설시장,도시 외각 지역의 규모가 큰 신설시장,"주차장을 보유한 5일장, 복합장",복합장 시장,편의시설이 없는 농산물 시장,화장실만 보유한 시장,주변 시설이 낙후된 시장,대형마트가 없는 관광지구 시장,"주유소,충전소가 많은 대형마트가 없는 비관광지구 시장",대중교통 이용이 편리한 시장,학교 주변 시장,"학교, 관광명소와 떨어진 문화시설 주변 시장",외각지역의 시장,주거지에서 벗어난 시장
0,1.776822,0.071416,0.166723,0.650700,1.081693,0.242413,-0.001944,-1.547419,-0.824616,0.196019,0.584936,-0.218246,0.205494,-2.771738,-0.509847
1,-0.667371,-0.993128,-0.577693,-0.331406,-0.012136,-0.463376,-0.112696,-0.303675,0.907540,-0.655388,0.067741,0.061151,0.949341,1.289843,-3.086018
2,-1.018561,-0.187543,-0.284827,1.500597,-1.333727,0.995525,0.407282,2.439395,-0.807369,0.644863,0.208599,-0.610732,0.333015,0.170042,-1.296120
3,-2.057687,0.793231,0.338318,1.500597,-1.333727,0.995525,0.407282,4.073832,-1.328353,0.958061,0.202246,-0.779758,0.942356,1.845876,1.632737
4,-1.519341,0.713279,-0.297835,1.500597,-1.333727,0.995525,0.407282,2.457932,-0.624651,-0.526818,0.023481,-0.325017,0.005182,0.577623,0.173966


In [ ]:
pca_mca.to_excel('../data/pca_mca_data.xlsx', index = False)